In [14]:
from openai import OpenAI
from openai import AssistantEventHandler


In [6]:
client = OpenAI()
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer math questions as requested",
    model="gpt-4o-mini"
)

In [8]:
assistant.id

'asst_eNp8MRrjH5W6JYY9YKV1TWqr'

In [7]:
thread = client.beta.threads.create()

In [9]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equations `3x+11 = 14`. Can you help me?"
)

In [10]:
response = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as John Doe, The user has premium account",
)

In [16]:
class EventHandler(AssistantEventHandler):    
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

In [17]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > Of course, Jane Doe! To solve the equation \(3x + 11 = 14\), follow these steps:

1. Subtract 11 from both sides of the equation:
   \[
   3x + 11 - 11 = 14 - 11
   \]
   This simplifies to:
   \[
   3x = 3
   \]

2. Now, divide each side by 3:
   \[
   \frac{3x}{3} = \frac{3}{3}
   \]
   This simplifies to:
   \[
   x = 1
   \]

So, the solution is \(x = 1\). If you have any further questions or need additional assistance, feel free to ask!